# Obtain Seyfert Samples
In this notebook we show the steps to obtain the Seyfert sample of galaxies.
We import the Python packages required to retrieve the data and create the match between the catalogs.

In [1]:
from datetime import date

import pyvo
import numpy as np
from astroquery.vizier import Vizier

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord, Angle

We define the date to save the date in the name of the files, as the data from the data archives can change with time.

In [2]:
today = date.today()

We set a the row limit to -1 to obtain all the rows in the catalog

In [3]:
Vizier.ROW_LIMIT = -1

## Obtaining the list
First, we connect through ``pyvo`` with the TAP Service of [SIMBAD](http://simbad.u-strasbg.fr/simbad/).Then, we obtain the SMB sample of galaxies (SMB_date.csv) with the following SQL query.
**Be aware that here we refer to SQL to the queries. However, [SIMBAD-TAP](http://simbad.u-strasbg.fr/simbad/sim-tap) actually uses [ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language), which is based on SQL.**
~~~~sql
SELECT main_id
	,ra
	,DEC
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [4]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,DEC
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

In [5]:
simbad_sample = result.to_table()
simbad_sample.write('../Data/Raw/SMB' + today.strftime('_%b_%d_%Y')+'.csv',
                    format='ascii.csv')

## Matching with VCV
First, we obtained the [Veron-Cetty M.P. , Veron P. 2010](https://cdsarc.unistra.fr/viz-bin/cat/VII/258) (VCV) catalog from [VizieR](https://vizier.u-strasbg.fr/viz-bin/VizieR) with ``astroquery``. Then, we cross-matched the two samples in terms of coordinates, assuming galaxies are the same if their coordinates differ by less than 2 arcsecs. The save the matched file as SMB_VCV_date.fits.

In [6]:
veron_cetty_veron = Vizier.get_catalogs('VII/258')[0]

From VCV we select only galaxies classified as Seyfert (all the ones that start with a S in the spectral type)

In [7]:
select_sy = [sp_type.startswith("S")
             for _, sp_type in enumerate(veron_cetty_veron['Sp'])]

We remove galaxies where their coordinates and redshifts is not described in SMB

In [8]:
bad_gal = np.where(np.logical_or.reduce([simbad_sample['ra'].mask,
                                        simbad_sample['dec'].mask,
                                        simbad_sample['rvz_redshift'].mask]))[0]
simbad_sample.remove_rows(bad_gal)

We rearrange the coordinates to match with SMB

In [9]:
vcv_sample = veron_cetty_veron[select_sy]
vcv_sample['ra'] = [Angle(vcv_sample['RAJ2000'][i],
                          unit='hourangle') for i in range(len(vcv_sample))]
vcv_sample['dec'] = [Angle(vcv_sample['DEJ2000'][i],
                           unit=u.deg) for i in range(len(vcv_sample))]

We perform the match between the two catalogs

In [10]:
cat1 = SkyCoord(ra=vcv_sample['ra'], dec=vcv_sample['dec'])
cat2 = SkyCoord(ra=simbad_sample['ra'], dec=simbad_sample['dec'])
idx, d2d, d3d = cat1.match_to_catalog_sky(cat2)

Then, we select only galaxies that are inside a 2D distance of 2 arcsecs. With this radius we do not find any multiple matches. 

In [11]:
sep_constraint = d2d <= 2*u.arcsec
vcv_match = vcv_sample[sep_constraint]
smb_match = simbad_sample[idx[sep_constraint]]

matched = hstack([smb_match, vcv_match['Name', 'ra', 'dec', 'z', 'Sp']])

Finally, we save the galaxies where the redshift is similar between the two catalogs.

In [12]:
matched.write('../Data/Raw/SMB_VCV'+today.strftime('_%b_%d_%Y')+'.csv', format='ascii.csv')

## Get the bibcodes for the classification type
We run the following SQL query to search for the bibcodes of the otypes in SMB with the SMBVCV_VO file.
~~~sql
SELECT DISTINCT main_id
	,ra
	,DEC
	,otypes.otype_txt
	,otypes.origin
FROM basic
JOIN ident ON oidref = oid
JOIN otypes USING (oidref)
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~
We save this information in a separated file (BibcodesOtypes.fits)

In [13]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

results_new = service.search("""
SELECT DISTINCT main_id
    ,ra
    ,DEC
    ,otypes.otype_txt
    ,otypes.origin
FROM basic
JOIN ident ON oidref = oid
JOIN otypes USING (oidref)
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""", maxrec=200000)

In [14]:
bibcodes_types = results_new.to_table()
bibcodes_types.write('../Data/Raw/BibcodesOtypes'+ today.strftime('_%b_%d_%Y')+'.csv',
                     format='ascii.csv')

In [15]:
%load_ext watermark
%watermark -a "Andres Ramos" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Andres Ramos

Python implementation: CPython
Python version       : 3.8.3
IPython version      : 7.16.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 3.10.0-1160.25.1.el7.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.2

astropy : 4.2
json    : 2.0.9
autopep8: 1.5.7
pyvo    : 1.1
numpy   : 1.19.5

Watermark: 2.1.0

